In [327]:
import numpy as np
import pandas as pd
import plotly.express as px
import colorsys
import parsedf
import json

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [262]:
def rgb_to_hex(rgb):
    return '%02x%02x%02x' % rgb

In [306]:
df_raw = pd.read_csv("static/files/thermoelectric_h2.csv", skiprows=8)
# df_raw = pd.read_csv("Channels_75bar_500ms_1mm.csv", skiprows=8)

In [308]:
df = df_raw.copy()
df = df.rename(columns={'% x': 'x', 'T (K)': 't'})
df_zrounded = pd.DataFrame(columns = ['x', 'y', 'z_rounded', 't'])
name_z = {'pipes': 2.5, 'te_bottom': 0.3, 'te_top': -2, 'only_pipes': 0}
for z_val in np.round(np.linspace(-2,2.5,46),1):
    z_min = z_val - 0.5
    z_max = z_val + 0.5
    df_xy = df.copy()
    df_xy = df_xy.loc[(df['z'] > z_min) & (df['z'] < z_max)]
    df_xy['z_rounded'] = z_val
    df_xy = df_xy[['x', 'y', 'z_rounded', 't']]
    df_zrounded = df_zrounded.append(df_xy)

In [392]:
df_xy = df_zrounded.copy()
# df_xy = df_xy.loc[df_xy['z_rounded'] == name_z['2.5']] # 2.5, 0.3, -2
# df_xy = df_xy.loc[df_xy['z_rounded'] == 0] # 2.5, 0.3, -2
df_xy = df_xy.loc[(df_xy['x'] < 150) & (df_xy['x'] >= 0)]
df_xy = df_xy.loc[(df_xy['y'] < 150) & (df_xy['y'] >= 0)]
df_xy = df_xy[['x', 'y', 'z_rounded', 't']]
num_y = 12
y_interval = np.floor((df_xy['y'].max() - df_xy['y'].min())/(num_y-1))
df_xy['y_round'] = df_xy[['y']]//y_interval
df_xy = df_xy.groupby(['y_round', 'z_rounded']).max().reset_index()
df_xy = df_xy[['y_round', 'z_rounded', 't']]
df_xy = df_xy.rename(columns={'y_round': 'y', 'z_rounded': 'z'})
# df_xy.to_pickle('static/files/thermoelectric_h2.pkl')

In [437]:
filename = 'thermoelectric_h2'
df_raw = pd.read_pickle(f'static/files/{filename}.pkl')
# df_xy = df_raw.loc[df_raw['z'] == 0].copy() # change Z depending on model
df_xy = df_raw.copy() # change Z depending on model
t_min = df_raw['t'].min()
t_max = df_raw['t'].max()
t_range = t_max - t_min

df_xy['t_normalized'] = (df_xy['t']-t_min)/t_range
df_xy['rgb'] = df_xy.apply(lambda x: colorsys.hsv_to_rgb(((1-x['t_normalized'])/360*240), 1, 1), axis=1)
df_xy['rgb_int'] = df_xy['rgb'].apply(lambda x: (int(np.floor(x[0]*255)), int(np.floor(x[1]*255)), int(np.floor(x[2]*255))))
df_xy['rgb_str'] = df_xy.apply(lambda x: f'rgb{x["rgb_int"]}', axis=1)
df_xy['rgb_tuple'] = df_xy.apply(lambda x: (x['t_normalized'], f'rgb{x["rgb_int"]}'), axis=1)

# df_xy['hex'] = df_xy['rgb_int'].apply(lambda x: f'#{rgb_to_hex(x)}')
df_final = pd.DataFrame(columns = ['x', 'y', 't'])
zs = [-2, 0.3, 2.5]
for z in zs:
    # for i in range(num_x):
    df_tmp = df_xy.loc[df_xy['z'] == z].copy()
    df_tmp['x'] = i
    df_final = df_final.append(df_tmp)


In [440]:
df_final

,x,y,t,z,t_normalized,rgb,rgb_int,rgb_str,rgb_tuple
45,4,0.0,294.162289,2.5,0.000000,"(0.0, 0.0, 1)","(0, 0, 255)","rgb(0, 0, 255)","(0.0, rgb(0, 0, 255))"
91,4,1.0,294.897817,2.5,0.038959,"(0.0, 0.15583496847005307, 1)","(0, 39, 255)","rgb(0, 39, 255)","(0.03895874211751337, rgb(0, 39, 255))"
137,4,2.0,295.235847,2.5,0.056863,"(0.0, 0.22745278584127648, 1)","(0, 58, 255)","rgb(0, 58, 255)","(0.05686319646031914, rgb(0, 58, 255))"
229,4,4.0,295.497066,2.5,0.070699,"(0.0, 0.2827966915726554, 1)","(0, 72, 255)","rgb(0, 72, 255)","(0.070699172893164, rgb(0, 72, 255))"
183,4,3.0,295.609256,2.5,0.076642,"(0.0, 0.30656613875299765, 1)","(0, 78, 255)","rgb(0, 78, 255)","(0.07664153468824934, rgb(0, 78, 255))"
321,4,6.0,295.836913,2.5,0.088700,"(0.0, 0.35479949458849447, 1)","(0, 90, 255)","rgb(0, 90, 255)","(0.08869987364712376, rgb(0, 90, 255))"
275,4,5.0,295.913212,2.5,0.092741,"(0.0, 0.3709648449800351, 1)","(0, 94, 255)","rgb(0, 94, 255)","(0.09274121124500864, rgb(0, 94, 255))"
413,4,8.0,296.157608,2.5,0.105686,"(0.0, 0.42274457730050763, 1)","(0, 107, 255)","rgb(0, 107, 255)","(0.10568614432512691, rgb(0, 107, 255))"
551,4,11.0,296.244935,2.5,0.110312,"(0.0, 0.4412463038936405, 1)","(0, 112, 255)","rgb(0, 112, 255)","(0.11031157597341, rgb(0, 112, 255))"
367,4,7.0,296.272107,2.5,0.111751,"(0.0, 0.4470031433290771, 1)","(0, 113, 255)","rgb(0, 113, 255)","(0.11175078583226938, rgb(0, 113, 255))"


In [448]:
colors = df_final['rgb_str'].tolist()
colors.reverse()
colors

['rgb(255, 0, 0)',
 'rgb(255, 0, 0)',
 'rgb(255, 5, 0)',
 'rgb(255, 10, 0)',
 'rgb(255, 10, 0)',
 'rgb(255, 12, 0)',
 'rgb(255, 22, 0)',
 'rgb(255, 30, 0)',
 'rgb(255, 38, 0)',
 'rgb(255, 52, 0)',
 'rgb(255, 69, 0)',
 'rgb(255, 82, 0)',
 'rgb(0, 255, 93)',
 'rgb(0, 255, 93)',
 'rgb(0, 255, 100)',
 'rgb(0, 255, 102)',
 'rgb(0, 255, 105)',
 'rgb(0, 255, 105)',
 'rgb(0, 255, 116)',
 'rgb(0, 255, 123)',
 'rgb(0, 255, 131)',
 'rgb(0, 255, 145)',
 'rgb(0, 255, 163)',
 'rgb(0, 255, 176)',
 'rgb(0, 123, 255)',
 'rgb(0, 122, 255)',
 'rgb(0, 113, 255)',
 'rgb(0, 112, 255)',
 'rgb(0, 107, 255)',
 'rgb(0, 94, 255)',
 'rgb(0, 90, 255)',
 'rgb(0, 78, 255)',
 'rgb(0, 72, 255)',
 'rgb(0, 58, 255)',
 'rgb(0, 39, 255)',
 'rgb(0, 0, 255)']

In [450]:

# fig = px.scatter_3d(df_final, x='x', y='y', z='z', color='t', color_continuous_scale=df_final['rgb_str'].tolist())
fig = px.scatter_3d(df_final, x='x', y='y', z='z', color='t', color_continuous_scale='hsv')

fig.update_layout({'paper_bgcolor': 'rgba(0,0,0,0)', 'plot_bgcolor': 'rgba(0,0,0,0)'})
fig.update_layout(title_text='Heat Map', title_x=0.5, title_y=0.925)
fig.show()

In [421]:
fig = px.scatter(df_final, x='x', y='y', color='t', color_continuous_scale=df_xy.loc[df_xy['z'] == 0]['rgb_str'].tolist())
fig.update_layout({'paper_bgcolor': 'rgba(0,0,0,0)', 'plot_bgcolor': 'rgba(0,0,0,0)'})
fig.update_layout(title_text='Heat Map', title_x=0.5, title_y=0.925)
fig.show()

In [419]:

layers = [0]
for i, layer in enumerate(layers):
    str = f'{i+1}:'
    for index, row in df_xy.loc[df_xy['z'] == layer]['rgb_int'].iteritems():
        r = row[0]
        g = row[1]
        b = row[2]
        str = str+f'{r},{g},{b},'
str = str[0:-1]
str


'1:30,255,0,146,255,0,212,255,0,255,242,0,255,195,0,255,155,0,255,115,0,255,88,0,255,69,0,255,40,0,255,18,0,255,0,0'

In [401]:
df_xy['rgb_int']

4       (12, 255, 0)
34      (22, 255, 0)
64      (69, 255, 0)
94     (130, 255, 0)
124    (237, 255, 0)
154    (255, 213, 0)
184    (255, 247, 0)
214    (255, 195, 0)
244    (255, 145, 0)
274     (255, 91, 0)
304      (255, 1, 0)
334      (255, 0, 0)
Name: rgb_int, dtype: object

In [14]:
import serial

In [15]:
# Importing Libraries

# arduino = serial.Serial(port='/dev/cu.usbmodem101', baudrate=9600, timeout=.1)

arduino = serial.Serial(port='/dev/cu.usbmodem101', baudrate=115200, timeout=.1)

# while True:
#     num = input("Enter a number: ") # Taking input from user
#     value = write_read(num)
#     print(value) # printing the value


In [16]:
x = '255,19,0,255,101,0,255,194,0,236,255,0,161,255,0,92,255,0,23,255,0,0,255,34,0,255,90,0,255,147,0,255,200,0,254,255,0,208,255,0,163,255,0,118,255,0,80,255,0,40,255,0,17,255,0,29,255'
arduino.write(bytes(x, 'utf-8'))

180

In [17]:
arduino.close()

In [128]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import colorsys
import parsedf

df_raw = pd.read_csv("final.csv", skiprows=8)
import parsedf

In [150]:
fig = parsedf.parse(df_raw, 5, 19, 2.5)
fig.show()

In [20]:
json.loads('{"0": "255,19,0,255,101,0,255,194,0,236,255,0,161,255,0,92,255,0,23,255,0,0,255,34,0,255,90,0,255,147,0,255,200,0,254,255,0,208,255,0,163,255,0,118,255,0,80,255,0,40,255,0,17,255,0,29,25", "1": "0,36,255,0,15,255,0,38,255,0,78,255,0,115,255,0,158,255,0,208,255,0,251,255,0,255,209,0,255,151,0,255,99,0,255,33,23,255,0,91,255,0,157,255,0,226,255,0,255,190,0,255,111,0,255,21,", "2": "255,0,0,255,87,0,255,178,0,249,255,0,179,255,0,109,255,0,45,255,0,0,255,17,0,255,77,0,255,134,0,255,183,0,255,234,0,229,255,0,176,255,0,138,255,0,99,255,0,67,255,0,37,255,0,53,25", "3": "0,32,255,0,22,255,0,45,255,0,77,255,0,123,255,0,160,255,0,207,255,0,252,255,0,255,202,0,255,151,0,255,91,0,255,37,29,255,0,90,255,0,164,255,0,236,255,0,255,187,0,255,103,0,255,18,", "4": "255,17,0,255,94,0,255,195,0,240,255,0,157,255,0,94,255,0,20,255,0,0,255,38,0,255,104,0,255,150,0,255,215,0,249,255,0,197,255,0,134,255,0,111,255,0,70,255,0,20,255,0,0,255,0,12,25"}')

{'0': '255,19,0,255,101,0,255,194,0,236,255,0,161,255,0,92,255,0,23,255,0,0,255,34,0,255,90,0,255,147,0,255,200,0,254,255,0,208,255,0,163,255,0,118,255,0,80,255,0,40,255,0,17,255,0,29,25',
 '1': '0,36,255,0,15,255,0,38,255,0,78,255,0,115,255,0,158,255,0,208,255,0,251,255,0,255,209,0,255,151,0,255,99,0,255,33,23,255,0,91,255,0,157,255,0,226,255,0,255,190,0,255,111,0,255,21,',
 '2': '255,0,0,255,87,0,255,178,0,249,255,0,179,255,0,109,255,0,45,255,0,0,255,17,0,255,77,0,255,134,0,255,183,0,255,234,0,229,255,0,176,255,0,138,255,0,99,255,0,67,255,0,37,255,0,53,25',
 '3': '0,32,255,0,22,255,0,45,255,0,77,255,0,123,255,0,160,255,0,207,255,0,252,255,0,255,202,0,255,151,0,255,91,0,255,37,29,255,0,90,255,0,164,255,0,236,255,0,255,187,0,255,103,0,255,18,',
 '4': '255,17,0,255,94,0,255,195,0,240,255,0,157,255,0,94,255,0,20,255,0,0,255,38,0,255,104,0,255,150,0,255,215,0,249,255,0,197,255,0,134,255,0,111,255,0,70,255,0,20,255,0,0,255,0,12,25'}

In [256]:
arr = ['94,50,168,' for x in range(12)]

In [257]:
tmp = ''.join(arr)
tmp

'94,50,168,94,50,168,94,50,168,94,50,168,94,50,168,94,50,168,94,50,168,94,50,168,94,50,168,94,50,168,94,50,168,94,50,168,'

In [23]:
res = '255,19,0,255,101,0,255,194,0,236,255,0,161,255,0,92,255,0,23,255,0,0,255,34,0,255,90,0,255,147,0,255,200,0,254,255,0,208,255,0,163,255,0,118,255,0,80,255,0,40,255,0,17,255,0,29,2550,36,255,0,15,255,0,38,255,0,78,255,0,115,255,0,158,255,0,208,255,0,251,255,0,255,209,0,255,151,0,255,99,0,255,33,23,255,0,91,255,0,157,255,0,226,255,0,255,190,0,255,111,0,255,21,0,255,0,0,255,87,0,255,17'

In [24]:
len(res)

384

In [25]:
tmp.count('60')
# str.count(',')

19